### **Pricing Movement Classifier Bot**

In [1]:
import pandas as pd
import numpy as np
from tradeAlpaca import tradeAlpaca
import random
import numpy as np
import yfinance as yf
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense 
#from tensorflow.keras.optimizers import Adam, RMSprop
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from catboost import Pool, CatBoostClassifier, cv, CatBoostRegressor
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.feature_selection import SelectFromModel
from matplotlib import style
import plotly.express as px 
import plotly.io as pio
# add this to yaml (coinbase pro library)
import cbpro
from datetime import datetime, timedelta
pio.renderers.default = "notebook_connected"
style.use('dark_background')
pd.set_option('display.max_columns', None)

In [2]:
# get historical data
trade_inst = tradeAlpaca(keys_file="../../data/alpaca_keys.cfg")
#trade_inst.get_current_price('SPY')

In [3]:
#!pip install cbpro --user

In [4]:
c = cbpro.PublicClient()
data_ = pd.DataFrame(c.get_products())
crypto_pairs = list(data_['id'].values)
crypto_pairs[:5]

['REQ-USD', 'GHST-USD', 'QNT-USD', 'RLY-USD', 'C98-USDT']

In [61]:
trade_inst.get_historical_prices('SPY', start=start_date, end=end_date, freq='minute')

open    high     low   close  volume  \
symbol timestamp                                                           
SPY    2023-08-11 08:00:00+00:00  446.25  446.31  446.25  446.31   957.0   
       2023-08-11 08:01:00+00:00  446.17  446.25  446.17  446.24  1141.0   
       2023-08-11 08:02:00+00:00  446.23  446.23  446.22  446.22   361.0   
       2023-08-11 08:03:00+00:00  446.19  446.19  446.19  446.19   176.0   
       2023-08-11 08:04:00+00:00  446.23  446.23  446.21  446.22   713.0   
...                                  ...     ...     ...     ...     ...   
       2023-09-07 23:55:00+00:00  445.12  445.12  445.08  445.08   711.0   
       2023-09-07 23:56:00+00:00  445.12  445.12  445.12  445.12   254.0   
       2023-09-07 23:57:00+00:00  445.05  445.05  445.05  445.05   326.0   
       2023-09-07 23:58:00+00:00  445.05  445.05  445.05  445.05  1195.0   
       2023-09-07 23:59:00+00:00  445.03  445.03  445.03  445.03  1670.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2023-08-11 08:00:00+00:00         14.0  446.278433  
       2023-08-11 08:01:00+00:00         10.0  446.212866  
       2023-08-11 08:02:00+00:00          7.0  446.219418  
       2023-08-11 08:03:00+00:00          2.0  446.172727  
       2023-08-11 08:04:00+00:00          4.0  446.224208  
...                                       ...         ...  
       2023-09-07 23:55:00+00:00          9.0  445.109339  
       2023-09-07 23:56:00+00:00         20.0  445.104409  
       2023-09-07 23:57:00+00:00         11.0  445.052454  
       2023-09-07 23:58:00+00:00         20.0  445.049701  
       2023-09-07 23:59:00+00:00         25.0  445.029964  

[15203 rows x 7 columns]

In [46]:
# start_date = '2023-08-11'
# end_date = '2023-09-08'
# interval = '1m'
# tickers = ['SPY']

# crypto_data_df = pd.DataFrame()
# # check if end_date - start_date is greater than 7 days and if interval == '1m'
# # if it is, then loop through 7 dates at a time and download that data
# if (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days > 7 and interval == '1m':
#     # Define the interval size (7 days)
#     interval = timedelta(days=7)

#     # Initialize the current_start_date to the initial start_date
#     current_start_date = pd.to_datetime(start_date)

#     # Loop through the date range in 7-day intervals
#     while current_start_date <= pd.to_datetime(end_date):
#         # Calculate the end_date_i for the current chunk
#         current_end_date = current_start_date + interval - timedelta(days=1)
        
#         # If the current_end_date is beyond the specified end_date, adjust it
#         if current_end_date > pd.to_datetime(end_date):
#             current_end_date = pd.to_datetime(end_date)
        
#         # Print the start_date_i and end_date_i for the current chunk
#         print(f"Start Date: {current_start_date.strftime('%Y-%m-%d')}, End Date: {current_end_date.strftime('%Y-%m-%d')}")
        
#         start_date_i = current_start_date.strftime('%Y-%m-%d')
#         end_date_i = current_end_date.strftime('%Y-%m-%d')

#         if start_date_i != end_date_i:
#             crypto_data = yf.download(tickers, start=start_date_i,
#                                 end=end_date_i, interval='1m', progress=False)
#         else:
#             crypto_data = yf.download(tickers, start=start_date_i,
#                                  interval='1m', progress=False)
#         if len(tickers) > 1:
#             crypto_data = crypto_data.stack(1).reset_index().rename(columns = {'level_1':'Ticker'})

#         crypto_data_df = pd.concat([crypto_data_df, crypto_data])
#         # Move the current_start_date to the next interval
#         current_start_date += interval
    
# crypto_data_df.head()

Start Date: 2023-08-11, End Date: 2023-08-17
Start Date: 2023-08-18, End Date: 2023-08-24
Start Date: 2023-08-25, End Date: 2023-08-31
Start Date: 2023-09-01, End Date: 2023-09-07
Start Date: 2023-09-08, End Date: 2023-09-08


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-08-11 09:30:00-04:00,443.970001,444.160004,443.739990,443.778992,443.778992,2185318
2023-08-11 09:31:00-04:00,443.769989,443.850098,443.390015,443.459991,443.459991,401451
2023-08-11 09:32:00-04:00,443.434998,443.649994,443.345001,443.625000,443.625000,262763
2023-08-11 09:33:00-04:00,443.625000,443.940002,443.570007,443.829987,443.829987,203539
2023-08-11 09:34:00-04:00,443.850006,444.160004,443.840088,444.079987,444.079987,242445


In [56]:
# pd.set_option('display.max_rows', 100)  # Set the maximum number of rows to 100

# crypto_data_df.head(100)

In [57]:

# 60*6.5*30 - crypto_data_df.shape[0]

# 60*6.5*30

11700.0

In [38]:
# expected_timestamps[0]


# pd.to_datetime(crypto_data_df['Datetime']).iloc[0]

Timestamp('2023-08-11 00:00:00+0000', tz='UTC')

In [39]:
# # Create a list of expected timestamps for the entire month
# # start_date = datetime(2023, 5, 1)
# # end_date = datetime(2023, 5, 31, 23, 59)
# expected_timestamps = pd.date_range(start=pd.to_datetime(start_date), end=pd.to_datetime(end_date), freq='T', tz='UTC')
# # Identify gaps by comparing expected timestamps with actual timestamps
# missing_timestamps = expected_timestamps[~expected_timestamps.isin(pd.to_datetime(crypto_data_df['Datetime']))]

# # Display the missing timestamps
# print("Missing timestamps:")
# print(missing_timestamps)

Missing timestamps:
DatetimeIndex(['2023-08-11 00:02:00+00:00', '2023-08-11 00:07:00+00:00',
               '2023-08-11 00:12:00+00:00', '2023-08-11 01:03:00+00:00',
               '2023-08-11 01:08:00+00:00', '2023-08-11 01:22:00+00:00',
               '2023-08-11 01:52:00+00:00', '2023-08-11 01:58:00+00:00',
               '2023-08-11 02:08:00+00:00', '2023-08-11 02:17:00+00:00',
               ...
               '2023-09-07 23:50:00+00:00', '2023-09-07 23:51:00+00:00',
               '2023-09-07 23:52:00+00:00', '2023-09-07 23:53:00+00:00',
               '2023-09-07 23:54:00+00:00', '2023-09-07 23:55:00+00:00',
               '2023-09-07 23:56:00+00:00', '2023-09-07 23:57:00+00:00',
               '2023-09-07 23:58:00+00:00', '2023-09-07 23:59:00+00:00'],
              dtype='datetime64[ns, UTC]', length=6855, freq=None)


In [26]:
# type(crypto_data_df.Datetime.iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [22]:
# crypto_data_df.shape[0]

71737

In [ ]:
# !pip install --upgrade six

In [ ]:
import yfinance as yf
import pandas as pd
from hurst import compute_Hc

def theil_sen_slope(x, y):
    n = len(x)
    slopes = []

    for i in range(n):
        for j in range(i + 1, n):
            if x[i] != x[j]:
                slopes.append((y[j] - y[i]) / (x[j] - x[i]))

    median_slope = np.median(slopes)
    return median_slope

c = cbpro.PublicClient()
data_ = pd.DataFrame(c.get_products())
crypto_pairs = list(data_['id'].values)
crypto_pairs[:5]

# List of cryptocurrencies to analyze (you can add more as needed)
cryptos = crypto_pairs

# Initialize an empty DataFrame to store results
#results = pd.DataFrame(columns=["Symbol", "Hurst_Exponent"])
results = []
no_data = []

# Define the time frame (1 week) and resolution (1 minute)
# start_date = "2023-08-01"
# end_date = "2023-09-02"
interval = "1m"

for crypto_symbol in cryptos:
    
    try:
        # Fetch historical minute-level data
        crypto_data = yf.download(crypto_symbol, period="5d", interval=interval, progress=False)

        if not crypto_data.empty:
            # Calculate the Hurst exponent
            H, c, data = compute_Hc(crypto_data["Close"].dropna(), simplified=True)

            # Extract the Hurst exponent value
            hurst_exponent = H

             # Calculate the Theil-Sen slope
            theil_sen = theil_sen_slope(np.arange(len(crypto_data)), crypto_data["Close"].values)

            # Determine the trend based on the Theil-Sen slope
            if theil_sen > 0:
                trend = "Upward"
            elif theil_sen < 0:
                trend = "Downward"
            else:
                trend = "Sideways"
            #crypto_data["SMA"] = crypto_data["Close"].rolling(window=50).mean()

            # Calculate the MACD line
            # short_window = 12
            # long_window = 26
            # crypto_data["Short_EMA"] = crypto_data["Close"].ewm(span=short_window, adjust=False).mean()
            # crypto_data["Long_EMA"] = crypto_data["Close"].ewm(span=long_window, adjust=False).mean()
            # crypto_data["MACD"] = crypto_data["Short_EMA"] - crypto_data["Long_EMA"]

            # # Determine the trend based on SMA and MACD
            # if crypto_data["Close"].iloc[-1] > crypto_data["SMA"].iloc[-1] and crypto_data["MACD"].iloc[-1] > 0:
            #     trend = "Upward"
            # elif crypto_data["Close"].iloc[-1] < crypto_data["SMA"].iloc[-1] and crypto_data["MACD"].iloc[-1] < 0:
            #     trend = "Downward"
            # else:
            #     trend = "Sideways"

            # Append the result to the DataFrame
            results.append({"Symbol": crypto_symbol, "Hurst_Exponent": hurst_exponent, "Trend": trend, "Theil-Sen": theil_sen})
        else:
            #print(f"No data found for {crypto_symbol}")
            no_data.append({"Symbol": crypto_symbol, "reason": "No data found"})
    except Exception as e:
        #print(f"Error encountered while processing {crypto_symbol}: {e}")
        no_data.append({"Symbol": crypto_symbol, "reason": e})

results_df = pd.DataFrame(results)
# Print the results
results_df.head()

In [ ]:
results_df.to_csv("20230902_last_week_crypto_trends.csv", index=False)

In [ ]:
results_df.query("Trend == 'Upward'").sort_values(['Hurst_Exponent','Theil-Sen'], ascending=False)

In [ ]:
# Lets analyze Nvidia. We know from quick lookup of Nvidia news
# that Intel performance is connected to its performance, in addition to AMD.
# (source: https://www.barrons.com/articles/nvidia-intels-report-is-a-great-leading-indicator-says-raymond-james-1525724415)
# NDX is the index for the technology sector.
tickers = ['^NDX', 'INTC', 'AMD', 'NVDA']
data = (yf.download(tickers, start='2010-01-01', end='2022-02-18')
            .drop(columns = ['Close','Open','High','Low','Volume'])
)
data.columns = data.columns.droplevel()

for i in tickers:
    data[f'{i}_log_returns'] = (data[i].pct_change() + 1).apply(np.log)
data.tail()
data.plot(y=['NVDA_log_returns','AMD_log_returns','INTC_log_returns','^NDX_log_returns'], 
        use_index=True, figsize=(10,6))
# log returns quickly and efficiently detrends the pricing history

In [ ]:
# lets define a direction column which is essentially 1 if the log return is positive and 0 if
# it is negative (classification tasks typically have a target variable which is 1 or 0)
for i in tickers:
    data[f'{i}_direction'] = np.where(data[f'{i}_log_returns'] > 0, 1, 0)

data.tail()

In [ ]:
# traditionally to determine optimal lags of the target variable, we look at the autocorrelation 
# and partial autocorrelation of the target variable
plot_acf(data['NVDA_direction'], lags=10)
plot_pacf(data['NVDA_direction'], lags=10)

# looking at these plots, lagging the target variable probably won't help us

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	features = data.columns
	df = data
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(f'{j}(t-{i})') for j in features]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(f'{j}') for j in features]
		else:
			names += [(f'{j}(t+{i})') for j in features]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# essentially build a forecast which predicts tomorrow. Excessive lagging of features but its fine because we can 
# examine the feature importance after modelling.
data = series_to_supervised(data)
data.tail()

In [ ]:
data.reset_index(inplace=True)
data['year'] = data.Date.dt.year
data['month'] = data.Date.dt.month 
data['day'] = data.Date.dt.day
data['week'] = data.Date.dt.isocalendar().week
data['quarterstart'] = data.Date.dt.is_quarter_start.astype(int)
data['quarterend'] = data.Date.dt.is_quarter_end.astype(int)
data['dow'] = data.Date.dt.dayofweek
data['quarter'] = data.Date.dt.quarter
data.set_index('Date', inplace=True)
data.head()

In [ ]:
trade_inst.get_historical_prices("SPY", start="2023-08-20", end="2023-09-02").head()